# Output Parsers for LLM Input / Output with LangChain

## Install OpenAI, HuggingFace and LangChain dependencies

In [3]:
!pip install -qq langchain==0.3.11
!pip install -qq langchain-openai==0.2.12
!pip install -qq langchain-community==0.3.11

In [23]:
!pip install -qq pandas

## Enter API Tokens

#### Enter your Open AI Key here

You can get the key from [here](https://platform.openai.com/api-keys) after creating an account or signing in

In [ ]:
# from getpass import getpass

# OPENAI_KEY = getpass('Please enter your Open AI API Key here: ')

In [ ]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Setup necessary system environment variables

## Chat Models and LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not implement or build its own LLMs. It provides a standard API for interacting with almost every LLM out there.

There are lots of LLM providers (OpenAI, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

## Accessing Commercial LLMs like ChatGPT

In [5]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Output Parsers
Output parsers are essential in Langchain for structuring the responses from language models. Below, we will discuss the role of output parsers and include examples using Langchain's specific parser types: PydanticOutputParser, JsonOutputParser, and CommaSeparatedListOutputParser.

- **Pydantic parser:**
  - This parser allows the specification of an arbitrary Pydantic Model to query LLMs for outputs matching that schema. Pydantic's BaseModel functions similarly to a Python dataclass but includes type checking and coercion.

- **JSON parser:**
  - Users can specify an arbitrary JSON schema with this parser to ensure outputs from LLMs adhere to that schema. Pydantic can also be used to declare your data model here.

- **CSV parser:**
  - Useful for outputs requiring a list of items separated by commas. This parser facilitates the extraction of comma-separated values from model outputs.


### 1 -> PydanticOutputParser

This output parser allows users to specify an arbitrary Pydantic Model and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are non-deterministic! You'll have to use an LLM with sufficient capacity to generate well-formed responses.

Use Pydantic to declare your data model. Pydantic's BaseModel is like a Python dataclass, but with actual type checking + coercion.

In [6]:
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
from typing import List

# Define your desired data structure - like a python data class.
class QueryResponse(BaseModel):
    description: str = Field(description="A brief description of the topic asked by the user")
    pros: str = Field(description="3 bullet points showing the pros of the topic asked by the user")
    cons: str = Field(description="3 bullet points showing the cons of the topic asked by the user")
    conclusion: str = Field(description="One line conclusion of the topic asked by the user")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=QueryResponse)
parser

PydanticOutputParser(pydantic_object=<class '__main__.QueryResponse'>)

In [7]:
# langchain pre-generated output response formatting instructions
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "A brief description of the topic asked by the user", "title": "Description", "type": "string"}, "pros": {"description": "3 bullet points showing the pros of the topic asked by the user", "title": "Pros", "type": "string"}, "cons": {"description": "3 bullet points showing the cons of the topic asked by the user", "title": "Cons", "type": "string"}, "conclusion": {"description": "One line conclusion of the topic asked by the user", "title": "Conclusion", "type": "string"}}, "required": ["descriptio

In [8]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Answer the user query and generate the response based on the following formatting instructions

             Format Instructions:
             {format_instructions}

             Query:
             {query}
            """

prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | chatgpt | parser

response = chain.invoke({"query": "What is the capital of France?"})
print(response)

description='The capital of France is Paris, known for its art, fashion, and culture.' pros='1. Paris is a major European city and a global center for art, fashion, and culture.\n2. It is home to famous landmarks such as the Eiffel Tower and the Louvre Museum.\n3. Paris offers a rich history and vibrant lifestyle, attracting millions of tourists each year.' cons='1. Paris can be very expensive in terms of living costs and tourism.\n2. The city often faces issues with traffic congestion and pollution.\n3. It can be crowded, especially during peak tourist seasons.' conclusion='Overall, Paris is a culturally rich and iconic city, despite some challenges.'


In [9]:
question = "Tell me about Commercial Real Estate"
response = chain.invoke({"query": question})

In [11]:
print(response.description)

Commercial real estate refers to properties used exclusively for business purposes, including office buildings, retail spaces, warehouses, and industrial properties.


In [12]:
print(response.pros)

1. Potential for high returns on investment.
2. Long-term leases provide stable income.
3. Diversification of investment portfolio.


In [13]:
response.dict()

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_2049/3202056457.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response.dict()


{'description': 'Commercial real estate refers to properties used exclusively for business purposes, including office buildings, retail spaces, warehouses, and industrial properties.',
 'pros': '1. Potential for high returns on investment.\n2. Long-term leases provide stable income.\n3. Diversification of investment portfolio.',
 'cons': '1. Requires significant capital investment.\n2. Market fluctuations can impact property values.\n3. Management and maintenance can be complex and time-consuming.',
 'conclusion': 'Commercial real estate can be a lucrative investment but comes with its own set of challenges.'}

In [14]:
for k,v in response.dict().items():
    print(f"{k}:\n{v}\n")

description:
Commercial real estate refers to properties used exclusively for business purposes, including office buildings, retail spaces, warehouses, and industrial properties.

pros:
1. Potential for high returns on investment.
2. Long-term leases provide stable income.
3. Diversification of investment portfolio.

cons:
1. Requires significant capital investment.
2. Market fluctuations can impact property values.
3. Management and maintenance can be complex and time-consuming.

conclusion:
Commercial real estate can be a lucrative investment but comes with its own set of challenges.



/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_2049/3926753719.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for k,v in response.dict().items():


### JsonOutputParser

This output parser allows users to specify an arbitrary JSON schema and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are non-deterministic! You'll have to use an LLM with sufficient capacity to generate well-formed responses.

It is recommended use Pydantic to declare your data model.


In [15]:
from typing import List

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# Define your desired data structure - like a python data class.
class QueryResponse(BaseModel):
    description: str = Field(description="A brief description of the topic asked by the user")
    pros: str = Field(description="3 bullet points showing the pros of the topic asked by the user")
    cons: str = Field(description="3 bullet points showing the cons of the topic asked by the user")
    conclusion: str = Field(description="One line conclusion of the topic asked by the user")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=QueryResponse)
parser

JsonOutputParser(pydantic_object=<class '__main__.QueryResponse'>)

In [17]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Answer the user query and generate the response based on the following formatting instructions

             Format Instructions:
             {format_instructions}

             Query:
             {query}
            """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"description": "A brief description of the topic asked by the user", "title": "Description", "type": "string"}, "pros": {"description": "3 bullet points showing the pros of the topic asked by the user", "title": "Pros", "type": "string"}, "cons": {"description": "3 bullet points showing the cons of the topic asked by the user", "title": "Cons", "type": "string"}, "conclusion": {"description": "One line conclusion of the to

In [18]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
chain = (prompt
              |
            chatgpt
              |
            parser)
chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"description": "A brief description of the topic asked by the user", "title": "Description", "type": "string"}, "pros": {"description": "3 bullet points showing the pros of the topic asked by the user", "title": "Pros", "type": "string"}, "cons": {"description": "3 bullet points showing the cons of the topic asked by the user", "title": "Cons", "type": "string"}, "conclusion": {"description": "One line con

In [19]:
topic_queries = [
    "Tell me about commercial real estate",
    "Tell me about Generative AI"
]

topic_queries_formatted = [{"query": topic}
                    for topic in topic_queries]
topic_queries_formatted

[{'query': 'Tell me about commercial real estate'},
 {'query': 'Tell me about Generative AI'}]

In [20]:
responses = chain.map().invoke(topic_queries_formatted)

In [21]:
responses[0], type(responses[0])

({'description': 'Commercial real estate refers to properties used exclusively for business purposes, including office buildings, retail spaces, warehouses, and industrial properties.',
  'pros': '1. Potential for high returns on investment.\n2. Long-term leases provide stable income.\n3. Diversification of investment portfolio.',
  'cons': '1. Requires significant capital investment.\n2. Market fluctuations can impact property values.\n3. Management and maintenance can be complex and time-consuming.',
  'conclusion': 'Investing in commercial real estate can be lucrative but comes with its own set of challenges.'},
 dict)

In [24]:
import pandas as pd

df = pd.DataFrame(responses)
df

,description,pros,cons,conclusion
0,Commercial real estate refers to properties us...,1. Potential for high returns on investment.\n...,1. Requires significant capital investment.\n2...,Investing in commercial real estate can be luc...
1,Generative AI refers to algorithms that can ge...,- Can create unique and diverse content quickl...,- May produce biased or inappropriate content ...,Generative AI holds great potential but requir...


In [25]:
for response in responses:
  for k,v in response.items():
    print(f"{k}:\n{v}\n")
  print('-----')

description:
Commercial real estate refers to properties used exclusively for business purposes, including office buildings, retail spaces, warehouses, and industrial properties.

pros:
1. Potential for high returns on investment.
2. Long-term leases provide stable income.
3. Diversification of investment portfolio.

cons:
1. Requires significant capital investment.
2. Market fluctuations can impact property values.
3. Management and maintenance can be complex and time-consuming.

conclusion:
Investing in commercial real estate can be lucrative but comes with its own set of challenges.

-----
description:
Generative AI refers to algorithms that can generate new content, such as text, images, music, and more, based on the data they have been trained on.

pros:
- Can create unique and diverse content quickly.
- Enhances creativity by providing new ideas and perspectives.
- Automates repetitive tasks, saving time and resources.

cons:
- May produce biased or inappropriate content if train

### CommaSeparatedListOutputParser

This output parser can be used when you want to return a list of comma-separated items.

In [27]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [34]:
format_instructions = output_parser.get_format_instructions()

# And a query intented to prompt a language model to populate the data structure.
prompt_txt = """
             Create a list of 5 different ways in which Generative AI can be used

             Output format instructions:
             {format_instructions}
             """

prompt = PromptTemplate.from_template(template=prompt_txt)

prompt = PromptTemplate(
    template=prompt_txt,
    partial_variables={"format_instructions": format_instructions},
)
print(prompt)

input_variables=[] input_types={} partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'} template='\n             Create a list of 5 different ways in which Generative AI can be used\n\n             Output format instructions:\n             {format_instructions}\n             '


In [35]:
# create a simple LLM Chain - more on this later
llm_chain = (prompt
              |
            chatgpt
              |
            output_parser)

# run the chain
response = llm_chain.invoke({})

In [38]:
print(response)

['Content creation', 'personalized marketing', 'virtual assistants', 'game design', 'data augmentation']


In [39]:
response

['Content creation',
 'personalized marketing',
 'virtual assistants',
 'game design',
 'data augmentation']

In [40]:
type(response)

list